In [4]:
import os
import time

import matplotlib.pyplot as plt
import timm_3d
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
from tqdm import tqdm
import logging
import seaborn as sn

from rsna_dataloader import *
from constants import *

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)


In [5]:
CONFIG = dict(
    n_levels=5,
    num_classes=25,
    num_conditions=5,
    image_interpolation="bspline",
    # backbone="maxvit_rmlp_nano_rw_256",
    backbone="coatnet_rmlp_narrow_rw",
    vol_size=(96, 96, 96),
    loss_weights=CLASS_LOGN_RELATIVE_WEIGHTS_MIRROR,
    num_workers=10,
    gradient_acc_steps=4,
    drop_rate=0,
    drop_rate_last=0.,
    drop_path_rate=0,
    aug_prob=0.9,
    out_dim=3,
    epochs=60,
    batch_size=5,
    split_rate=0.25,
    split_k=5,
    device=torch.device("cuda") if torch.cuda.is_available() else "cpu",
    seed=2024
)
DATA_BASEPATH = "../data/rsna-2024-lumbar-spine-degenerative-classification/"
TRAINING_DATA = retrieve_coordinate_training_data(DATA_BASEPATH)

filtered_df = pd.DataFrame(columns=TRAINING_DATA.columns)
for series_desc in CONDITIONS.keys():
    subset = TRAINING_DATA[TRAINING_DATA['series_description'] == series_desc]
    if series_desc == "Sagittal T2/STIR":
        subset = subset[subset.groupby(["study_id"]).transform('size') == 5]
    else:
        subset = subset[subset.groupby(["study_id"]).transform('size') == 10]
    filtered_df = pd.concat([filtered_df, subset])

TRAINING_DATA = filtered_df[filtered_df.groupby(["study_id"]).transform('size') == 25]

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [6]:
import torch.nn as nn
import timm_3d
from spacecutter import *
from spacecutter.losses import *
from spacecutter.models import *
from spacecutter.callbacks import *

from timm_3d.models import MaxxVitCfg
from timm_3d.models.maxxvit import _rw_coat_cfg


class CustomMaxxVit3dClassifier(nn.Module):
    def __init__(self,
                 backbone,
                 in_chans=3,
                 out_classes=5,
                 cutpoint_margin=0):
        super(CustomMaxxVit3dClassifier, self).__init__()
        self.out_classes = out_classes

        # self.config = timm_3d.models.maxxvit.model_cfgs[backbone]

        self.backbone = timm_3d.models.MaxxVit(
            img_size=CONFIG["vol_size"],
            in_chans=in_chans,
            num_classes=out_classes,
            drop_rate=CONFIG["drop_rate"],
            drop_path_rate=CONFIG["drop_path_rate"],
            cfg=MaxxVitCfg(
                embed_dim=(64, 256, 512, 1024),
                # embed_dim=(256, 512, 1280, 2048),
                depths=(2, 16, 32, 2),
                # stem_width=(128, 256),
                stem_width=(32, 64),
                **_rw_coat_cfg(
                    stride_mode='dw',
                    conv_attn_act_layer='silu',
                    init_values=1e-6,
                    rel_pos_type='mlp',
                    # rel_pos_dim=2048,
                ),
            )
        )
        self.backbone.head.drop = nn.Dropout(p=CONFIG["drop_rate_last"])
        head_in_dim = self.backbone.head.fc.in_features
        self.backbone.head.fc = nn.Identity()

        self.heads = nn.ModuleList(
            [nn.Sequential(
                nn.Linear(head_in_dim, 1),
                LogisticCumulativeLink(CONFIG["out_dim"])
            ) for i in range(out_classes)]
        )

        self.ascension_callback = AscensionCallback(margin=cutpoint_margin)

    def forward(self, x):
        feat = self.backbone(x)
        return torch.swapaxes(torch.stack([head(feat) for head in self.heads]), 0, 1)

    def _ascension_callback(self):
        for head in self.heads:
            self.ascension_callback.clip(head[-1])


In [33]:
import torch

model = CustomMaxxVit3dClassifier(backbone=CONFIG["backbone"]).to(device)
model.load_state_dict(torch.load("../models/coatnet_rmlp_narrow_rw_96_25_nonaligned_fold_1/coatnet_rmlp_narrow_rw_96_25_nonaligned_fold_1_0.pt"))
model.eval()

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


CustomMaxxVit3dClassifier(
  (backbone): MaxxVit(
    (stem): Stem(
      (conv1): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (norm1): BatchNormAct3d(
        32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
      (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (stages): Sequential(
      (0): MaxxVitStage(
        (blocks): Sequential(
          (0): MbConvBlock(
            (shortcut): Downsample3d(
              (pool): AvgPool3d(kernel_size=2, stride=2, padding=0)
              (expand): Identity()
            )
            (pre_norm): BatchNormAct3d(
              64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): SiLU(inplace=True)
            )
            (down): Identity()
            (conv1_1x1): Conv3d(64, 256, kern

In [34]:
bounds_dataframe = pd.read_csv(os.path.join("../data/SpineNet/bounding_boxes_3d.csv"))

In [35]:
coords_dataframe = pd.read_csv(os.path.join("../data/SpineNet/coords_3d.csv"))

In [36]:
import numpy as np

conditions = sorted(TRAINING_DATA["condition"].unique())

transform_3d_val = tio.Compose([
    tio.RescaleIntensity((0, 1)),
])

train_data = TRAINING_DATA[TRAINING_DATA["study_id"].isin(bounds_dataframe["study_id"])]
folds = create_vertebra_level_datasets_and_loaders_k_fold(train_data,
                                                          bounds_dataframe,
                                                          coords_df=coords_dataframe,
                                                            transform_3d_train=transform_3d_val,
                                                            transform_3d_val=transform_3d_val,
                                                            base_path=os.path.join(
                                                                DATA_BASEPATH,
                                                                "train_images"),
                                                            vol_size=(96, 96, 96),
                                                            num_workers=0,
                                                            split_k=5,
                                                            batch_size=1,
                                                            use_mirroring_trick=True)

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [37]:
trainloader, valloader, trainset, valset = folds[0]

In [ ]:
from torch.cuda.amp import autocast
from tqdm import tqdm

inferred = []
target = []

with torch.no_grad():
    with autocast(dtype=torch.bfloat16):
        for image, label in tqdm(valloader):
            target.append(label.detach())
            inferred.append(model(image.to(device)).cpu().detach())

`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
 83%|█████████████████████████████████████████████████████████████████▎             | 1448/1750 [03:28<00:41,  7.35it/s]

In [ ]:
inferred_new = []
target_new = []
for e in inferred:
    for elem in e:
        inferred_new.append(elem.reshape((-1, 5, 3)))
for e in target:
    for elem in e:
        target_new.append([[0 if i != elem_ else 1 for i in range(3)] for elem_ in elem])
        
        
inferred_new[1], target_new[1]

In [ ]:
inferred_l = [[np.argmax(e_) for e_ in e.detach().cpu()[0]] for e in inferred_new]
target_l = [[np.argmax(e_) for e_ in e] for e in target_new]

In [ ]:
len(inferred_l[1]), len(target_l[1])

In [ ]:
conditions

In [ ]:
import numpy as np
import seaborn as sn
from sklearn.metrics import confusion_matrix
from operator import truediv

precisions = []
recalls = []

fig, ax = plt.subplots(nrows=5, ncols=1, squeeze=False)
fig.tight_layout()
for i in range(5):
    target_ = [e[i] for e in target_l]
    inferred_ = [e[i] for e in inferred_l]
    
    ax[i][0].set_title(conditions[i])
    
    cf_matrix = confusion_matrix(target_, inferred_)
    sn.heatmap(cf_matrix / (np.sum(cf_matrix) + 1e-7), ax=ax[i][0], annot=True, fmt='.3%')

    true_pos = np.diag(cf_matrix)
    prec = list(map(truediv, true_pos, np.sum(cf_matrix, axis=0) + 1e-7))
    rec = list(map(truediv, true_pos, np.sum(cf_matrix, axis=1) + 1e-7))
    
    precisions.append(prec)
    recalls.append(rec)

plt.show()


In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=1)
fig.tight_layout()

plt.set_loglevel('WARNING')
for i in range(5):
    try:
        val_len = len(precisions[i])
        ax[i].set(ylim=(0, 1))
        
        ax[i].set_title(conditions[i])
        sn.barplot(x=range(val_len), y=precisions[i], ax=ax[i], label='Precision')
        sn.barplot(x=range(val_len, val_len * 2), y=recalls[i], ax=ax[i], label='Recall')
        bar_ax = sn.barplot(x=range(val_len * 2, val_len * 3), y=2 / (1 / (np.array(recalls[i]) + 1e-7) + 1 / (np.array(precisions[i]) + 1e-7)), ax=ax[i], label="F1")
        for i in bar_ax.containers:
           bar_ax.bar_label(i,)
    except:
        pass

plt.show()

In [ ]:
import timm_3d
timm_3d.list_models()